# E29AHU Satellite doppler calculation study
### - (Project Name: "HamSatPy")  email: e29ahu@gmail.com
### - Final goal to used with IC-705 and Pi Zero 2W for stand alone amateuradio sattelite comunication
<hr/>

### 1. Requesting SQF data from the github repository and extracting the data for satellite name, frequency and mode selection

In [56]:
# Requesting SQF data from the github repository and extracting the data for satellite name, frequency and mode selection
import requests
def req_satlist():
    tle_url = 'https://raw.githubusercontent.com/chokelive/HamSatTune/refs/heads/main/Doppler.sqf'
    response = requests.get(tle_url)
    ret = ""

    if response.status_code == 200:
        with open('Doppler.sqf', 'wb') as file:
            file.write(response.content)

    with open('Doppler.sqf', 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            ret += line

    return(ret)

print(req_satlist())

FO-29,435850,145950,USB,LSB,REV,0,0,SSB
FO-29,435850,145950,CW,CW,REV,0,0,CW
FO-99,437075,0,FM,FM,NOR,0,0,CW/SSTV
FO-99,435925,145915,CW,CW,REV,0,0,CW/SSB
HO-113,435180,145871,USB,LSB,REV,0,0,V/U SSB/CW
IO-86,435880,145880,FM,FM,NOR,0,0,FM tone 88.5Hz
IO-86,145825,145825,FM,FM,NOR,0,0,APRS
ISS,145825,145825,FM,FM,NOR,0,0,APRS
ISS,437800,145990,FM,FM,NOR,0,0,FM tone 67.0Hz
ISS,145800,0,FM,FM,NOR,0,0,SSTV
RS-44,435640,145964,USB,LSB,REV,0,0,SSB
RS-44,435640,145964,CW,CW,REV,0,0,CW
UVSQ-SAT,437020,145905,FM,FM,NOR,0,0,FM
SO-50,436795,145850,FM,FM,NOR,0,0,FM tone 67.0Hz
AO-27,436795,145850,FM,FM,NOR,0,0,FM
IO-117,435310,435310,DATA-USB,DATA-USB,NOR,0,0,DATA-USB
FO-118,435540,145820,USB,LSB,REV,0,0,VHF/UHF SSB
FO-118,435600,145925,FM,FM,NOR,0,0,VHF/UHF FM
Tevel-1,436400,145970,FM,FM,NOR,0,0,VHF/UHF FM
Tevel-2,436400,145970,FM,FM,NOR,0,0,VHF/UHF FM
Tevel-3,436400,145970,FM,FM,NOR,0,0,VHF/UHF FM
Tevel-4,436400,145970,FM,FM,NOR,0,0,VHF/UHF FM
Tevel-5,436400,145970,FM,FM,NOR,0,0,VHF/UHF FM
Teve

### 2. Identify the satellite name, frequency and mode and grid locator

In [57]:
## Identify the satellite name, frequency and mode and grid locator
sat_name = 'RS-44'
sat_mode = 'SSB'
grid_locator = 'NK93vt'

### 3. Request the TLE file from the AMSAT website and extracting the TLE data by satellite name selected 

In [58]:
## Request the TLE file from the AMSAT website and extracting the TLE data by satellite name selected 
def read_tle(satname):
    tle_url = 'https://www.amsat.org/tle/current/nasabare.txt'
    response = requests.get(tle_url)

    if response.status_code == 200:
        with open('tle.txt', 'wb') as file:
            file.write(response.content)

    with open('tle.txt', 'r') as file:
        lines = file.readlines()
        tle = ""
        for i, line in enumerate(lines):
            if satname in line:
                tle = lines[i] + lines[i+1] + lines[i+2]
                break
    return(tle)

print(read_tle(sat_name))


RS-44
1 44909U 19096E   24306.55452545  .00000043  00000-0  11763-3 0  9999
2 44909  82.5260  13.4920 0218501  90.0763 272.5375 12.79735466226542



### 4. Read satellite frequency from SQF file by satellite name and mode selected.

In [59]:
# Read satellite frequency from SQF file by satellite name and mode selected.
def read_sqf(satname, satemode):
    with open('Doppler.sqf', 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            line_arr = line.split(',')
            if len(line_arr) >= 8:
                if satname in line_arr[0] and satemode in line_arr[8]:
                    ret = line
                    break

    return(ret)

sqf_data = read_sqf(sat_name, sat_mode)
print(sqf_data)

RS-44,435640,145964,USB,LSB,REV,0,0,SSB



### 5. Calculate doppler shift frequency from TLE data

In [60]:
# Calculate doppler shift frequency from TLE data

import predict
from maidenhead import to_location
import json

def predict_sat(sat_name, grid_locator):
    tle = read_tle(sat_name)
    lat, lon = to_location(grid_locator)
    qth = (lat, lon, 7)
    sat = predict.observe(tle, qth) 

    return(sat)

sat_data = predict_sat(sat_name, grid_locator)
sat_data


{'norad_id': 44909,
 'name': 'RS-44',
 'epoch': 1730550930.2757812,
 'latitude': 27.75982174236604,
 'longitude': 317.754379597622,
 'azimuth': 65.65047152876343,
 'elevation': -17.68602045222495,
 'orbital_velocity': 26132.140388417978,
 'footprint': 7455.669822031897,
 'altitude': 1269.4579123547537,
 'slant_range': 6579.95867774096,
 'eclipse_depth': -72.5039748330706,
 'orbital_phase': 44.070471947209185,
 'orbital_model': 'SGP4',
 'visibility': b'D',
 'sunlit': 1,
 'orbit': 22667,
 'geostationary': 0,
 'has_aos': 1,
 'decayed': 0,
 'doppler': -107.04803287973687,
 'eci_x': -6689.835964048329,
 'eci_y': -1048.6028266682886,
 'eci_z': 3544.24927294821,
 'eci_vx': -3.255012261042674,
 'eci_vy': -1.5883910886604808,
 'eci_vz': -6.290782382129228,
 'eci_sun_x': -112715008.57603188,
 'eci_sun_y': -88595797.03354403,
 'eci_sun_z': -38409708.52778442,
 'eci_obs_x': -4097.438565021883,
 'eci_obs_y': 4646.977638681881,
 'eci_obs_z': 1510.6084273166553,
 'beta_angle': 22.085920392207004}

### 6. Calculate the satellite frequency doppler

To Calculate TX from RX Tuning, We use this equation:

- Setup IC-705 in Split Mode
- RX in VFO A
- TX in VFO B


RX_Tune = RX_Origin + doppler

TX_Tune = TX_Origin - doppler 

TX_Tune_LinearSat = TX_Origin + RX_Offset_From_RX_Origin


In [61]:
# Calculate the satellite frequency doppler
# ISS,437800,145990,FM,FM,NOR,0,0,FM tone 67.0Hz
def calc_txrx_doppler(rx_freq_origin, tx_freq_origin, doppler):
    rx_freq_tune = rx_freq_origin + doppler
    tx_freq_tune = tx_freq_origin - doppler
            

    return(rx_freq_tune, tx_freq_tune)

rx_freq_origin = int(sqf_data.split(',')[1]) * 1000
tx_freq_origin = int(sqf_data.split(',')[2]) * 1000
rx_freq_previous = rx_freq_origin
rx_freq_freetune = rx_freq_origin
transponderType = sqf_data.split(',')[5]
doppler = int(sat_data['doppler'])

rx_freq_tune, tx_freq_tune = calc_txrx_doppler(rx_freq_origin, tx_freq_origin, doppler)

print(rx_freq_tune, tx_freq_tune)

# Set rx tx frequency to RIG
# Finish doppler calculation

435639893 145964107



### Finish doppler calculation...
<hr/>

### Next step is special option, we can free tune RX frequency and software will recalculate Doppler 
### 7. For FM satellite, If user changed the RX frequency, We recalculate only RX Frequency + Doppler 

In [62]:
# for FM satellite, we allow to fine tune and recalculabe the doppler
flagFreeTune = False

rx_freq_tune = rx_freq_tune + 1500 # Read This frequency from RIG, we simulate ty tune 1.5 kHz up
if(abs(rx_freq_tune - rx_freq_previous > 50)): # If true mean the frequency is changed by user
   rx_freq_freetune = rx_freq_tune
   flagFreeTune = True

rx_freq_previous = rx_freq_freetune # get this frequency from RIG

# and we will use new rx_freq_freetune to calculate the doppler at next round by check flag "flagFreeTune"
rx_freq_tune, tx_freq_tune = calc_txrx_doppler(rx_freq_freetune, tx_freq_origin, doppler)

print(rx_freq_tune, tx_freq_tune)
# and save frequency to RIG

435641286 145964107


### 8. For linear satellite, If user changed the RX frequency, We recalculate both (RX and TX) Doppler  

In [ ]:
# For linear Satellite, we need calculate frequency offset duting fine tuning
tx_freq_tune = tx_freq_origin
rx_freq_tune_offset = int(rx_freq_origin - rx_freq_tune)

if (transponderType == "REV"):
        tx_freq_freetune = tx_freq_tune + rx_freq_tune_offset

# and we will use new rx_freq_freetune to calculate the doppler at next round by check flag "flagFreeTune"
rx_freq_tune, tx_freq_tune = calc_txrx_doppler(rx_freq_freetune, tx_freq_freetune, doppler)

print(rx_freq_tune, tx_freq_tune)
# and save frequency to RIG

435641286 145962821
